In [1]:
from dark_vessel_processor import DarkVesselProcessor
import pandas as pd

df = pd.read_pickle("xview3_multi.pkl")
# from list of dict to dataframe
df = pd.json_normalize(df)

BASE_GRD = "/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/GRD"
BASE_SLC = "/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/SLC"
BASE_L0 = "/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/L0"

# Initialize processor
processor = DarkVesselProcessor(
    base_directory='/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output',
    tile_size=512
)

# Load vessel validation data
vessels = None #processor.load_vessel_data('/path/to/vessel_validation.csv')

for index, row in df.iterrows():
    print(f"Processing scene {index + 1}/{len(df)}: {row['GRD']}")
    
    # Process scene
    success = processor.process_scene(
        grd_product=f"{BASE_GRD}/{row['GRD']}",
        slc_product=f"{BASE_SLC}/{row['SLC']}",
        vessel_data=vessels,
        tile_id=index + 1
    )
    if not success:
        print(f"Failed to process scene: {row['GRD']}")

    if index == 0:
        break  # Remove this line to process all scenes


INFO:dark_vessel_processor:No vessel data provided for tile 1, skipping metadata generation


Processing scene 1/699: S1A_IW_GRDH_1SDV_20201006T064054_20201006T064119_034672_0409DA_8975.SAFE


In [2]:
success

True

In [3]:
# Test the updated processor
print("Testing updated dark vessel processor...")

# Check if the first data entry exists
if len(df) > 0:
    first_row = df.iloc[0]
    print(f"First SLC product: {first_row['SLC']}")
    print(f"SLC path: {BASE_SLC}/{first_row['SLC']}")
    
    # Check if the SLC file exists
    slc_path = f"{BASE_SLC}/{first_row['SLC']}"
    import os
    if os.path.exists(slc_path):
        print("✓ SLC file exists")
    else:
        print("✗ SLC file not found")
else:
    print("No data found in dataframe")

Testing updated dark vessel processor...
First SLC product: S1A_IW_SLC__1SDV_20201006T064117_20201006T064145_034672_0409DA_706D.SAFE
SLC path: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/SLC/S1A_IW_SLC__1SDV_20201006T064117_20201006T064145_034672_0409DA_706D.SAFE
✓ SLC file exists


In [4]:
# Reload the updated processor module
import importlib
import dark_vessel_processor
importlib.reload(dark_vessel_processor)
from dark_vessel_processor import DarkVesselProcessor

# Re-initialize processor with updated code
processor = DarkVesselProcessor(
    base_directory='/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output',
    tile_size=512
)

# Test with the first SLC product
first_row = df.iloc[0]
print(f"Testing with SLC: {first_row['SLC']}")

try:
    success = processor.process_scene(
        grd_product="",  # Not using GRD
        slc_product=f"{BASE_SLC}/{first_row['SLC']}",
        vessel_data=None,  # No vessel data
        tile_id=1
    )
    
    if success:
        print("✓ Processing succeeded!")
    else:
        print("✗ Processing failed")
        
except Exception as e:
    print(f"Error during processing: {e}")
    import traceback
    traceback.print_exc()

INFO:dark_vessel_processor:No vessel data provided for tile 1, skipping metadata generation


Testing with SLC: S1A_IW_SLC__1SDV_20201006T064117_20201006T064145_034672_0409DA_706D.SAFE
✓ Processing succeeded!


In [5]:
# Check what files were generated
import os
output_base = '/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output'

print("Generated files:")
for subdir in ['slcfile', 'grdfile', 'xmlfile']:
    subdir_path = os.path.join(output_base, subdir)
    if os.path.exists(subdir_path):
        files = os.listdir(subdir_path)
        print(f"\n{subdir}:")
        for file in sorted(files):
            file_path = os.path.join(subdir_path, file)
            size = os.path.getsize(file_path)
            print(f"  {file} ({size} bytes)")
    else:
        print(f"\n{subdir}: Directory not found")

# Let's also check if we can inspect the manifest of the SLC file
slc_path = f"{BASE_SLC}/{first_row['SLC']}"
manifest_info = processor.parse_safe_manifest(slc_path)
print(f"\nManifest info for {first_row['SLC']}:")
print(f"Measurement files: {len(manifest_info['files']['measurement'])}")
print(f"Calibration files: {len(manifest_info['files']['calibration'])}")
print(f"Annotation files: {len(manifest_info['files']['annotation'])}")
print(f"Polarizations: {manifest_info['polarizations']}")

if manifest_info['files']['measurement']:
    print(f"\nFirst measurement file: {manifest_info['files']['measurement'][0]}")
if manifest_info['files']['calibration']:
    print(f"First calibration file: {manifest_info['files']['calibration'][0]}")

Generated files:

slcfile:

grdfile:

xmlfile:

Manifest info for S1A_IW_SLC__1SDV_20201006T064117_20201006T064145_034672_0409DA_706D.SAFE:
Measurement files: 0
Calibration files: 0
Annotation files: 0
Polarizations: []


In [6]:
# Let's examine the manifest.safe file directly
import xml.etree.ElementTree as ET

slc_path = f"{BASE_SLC}/{first_row['SLC']}"
manifest_path = os.path.join(slc_path, 'manifest.safe')

print(f"Checking manifest file: {manifest_path}")
print(f"Manifest exists: {os.path.exists(manifest_path)}")

if os.path.exists(manifest_path):
    # Read and examine the XML structure
    try:
        tree = ET.parse(manifest_path)
        root = tree.getroot()
        
        print(f"Root tag: {root.tag}")
        print(f"Root attributes: {root.attrib}")
        print(f"Namespaces in root: {list(root.nsmap.items()) if hasattr(root, 'nsmap') else 'N/A'}")
        
        # Look for dataObject elements
        print("\nLooking for dataObject elements...")
        data_objects = root.findall('.//*')
        print(f"Total elements found: {len(data_objects)}")
        
        # Look for elements containing 'dataObject'
        for elem in data_objects[:10]:  # Show first 10
            if 'dataObject' in elem.tag:
                print(f"Found dataObject: {elem.tag}")
                break
        
        # Try different namespace approaches
        print("\nTrying to find measurement files...")
        
        # Look for elements with 'measurement' in href
        for elem in data_objects:
            if elem.tag.endswith('fileLocation'):
                href = elem.get('href', '')
                if 'measurement' in href:
                    print(f"Found measurement file: {href}")
                    break
    
    except Exception as e:
        print(f"Error parsing manifest: {e}")
        
        # Try reading as text to see the structure
        with open(manifest_path, 'r') as f:
            content = f.read()
            print(f"First 1000 characters of manifest:")
            print(content[:1000])

Checking manifest file: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/SLC/S1A_IW_SLC__1SDV_20201006T064117_20201006T064145_034672_0409DA_706D.SAFE/manifest.safe
Manifest exists: True
Root tag: {urn:ccsds:schema:xfdu:1}XFDU
Root attributes: {'version': 'esa/safe/sentinel-1.0/sentinel-1/sar/level-1/slc/standard/iwdp'}
Namespaces in root: N/A

Looking for dataObject elements...
Total elements found: 319
Found dataObject: dataObjectPointer

Trying to find measurement files...
Found measurement file: ./measurement/s1a-iw1-slc-vh-20201006t064118-20201006t064143-034672-0409da-001.tiff


In [7]:
# Test the updated manifest parsing
import importlib
importlib.reload(dark_vessel_processor)
from dark_vessel_processor import DarkVesselProcessor

# Re-initialize processor with updated code
processor = DarkVesselProcessor(
    base_directory='/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output',
    tile_size=512
)

# Test manifest parsing
slc_path = f"{BASE_SLC}/{first_row['SLC']}"
manifest_info = processor.parse_safe_manifest(slc_path)

print(f"Updated manifest parsing results:")
print(f"Measurement files: {len(manifest_info['files']['measurement'])}")
for i, mf in enumerate(manifest_info['files']['measurement'][:3]):  # Show first 3
    print(f"  {i+1}: {mf}")

print(f"Calibration files: {len(manifest_info['files']['calibration'])}")
for i, cf in enumerate(manifest_info['files']['calibration'][:3]):  # Show first 3
    print(f"  {i+1}: {cf}")

print(f"Polarizations: {manifest_info['polarizations']}")

# Test processing again
print(f"\nTesting processing with updated manifest parsing...")
success = processor.process_scene(
    grd_product="",  # Not using GRD
    slc_product=slc_path,
    vessel_data=None,  # No vessel data
    tile_id=2
)

if success:
    print("✓ Processing succeeded with updated manifest parsing!")
else:
    print("✗ Processing failed")

INFO:dark_vessel_processor:Parsed manifest: 6 measurement files, 12 calibration files
INFO:dark_vessel_processor:Parsed manifest: 6 measurement files, 12 calibration files


Updated manifest parsing results:
Measurement files: 6
  1: measurement/s1a-iw3-slc-vh-20201006t064117-20201006t064145-034672-0409da-003.tiff
  2: measurement/s1a-iw2-slc-vv-20201006t064119-20201006t064144-034672-0409da-005.tiff
  3: measurement/s1a-iw2-slc-vh-20201006t064119-20201006t064144-034672-0409da-002.tiff
Calibration files: 12
  1: annotation/calibration/noise-s1a-iw2-slc-vh-20201006t064119-20201006t064144-034672-0409da-002.xml
  2: annotation/calibration/calibration-s1a-iw2-slc-vh-20201006t064119-20201006t064144-034672-0409da-002.xml
  3: annotation/calibration/calibration-s1a-iw3-slc-vv-20201006t064117-20201006t064145-034672-0409da-006.xml
Polarizations: ['VV', 'VH']

Testing processing with updated manifest parsing...


/Data_large/marine/PythonProjects/SAR/sarpyx/.venv/lib/python3.11/site-packages/rasterio/__init__.py:366: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = writer(
INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_2_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC VV tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_2_SLC_VV.tiff
INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_2_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC VV tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_2_SLC_VV.tiff
INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_2_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC 

✓ Processing succeeded with updated manifest parsing!


In [ ]:
# Check the generated files
output_base = '/Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output'

print("Generated files after updated processing:")
for subdir in ['slcfile', 'grdfile', 'xmlfile']:
    subdir_path = os.path.join(output_base, subdir)
    if os.path.exists(subdir_path):
        files = os.listdir(subdir_path)
        print(f"\n{subdir}:")
        for file in sorted(files):
            file_path = os.path.join(subdir_path, file)
            size = os.path.getsize(file_path)
            print(f"  {file} ({size} bytes)")
    else:
        print(f"\n{subdir}: Directory not found")

# Let's also try the scene tiling method
print(f"\n" + "="*50)
print("Testing scene tiling method...")

try:
    generated_tiles = processor.generate_scene_tiles(
        slc_safe_path=slc_path,
        vessel_data=None
    )
    print(f"Generated {len(generated_tiles)} tiles: {generated_tiles}")
except Exception as e:
    print(f"Error in scene tiling: {e}")
    import traceback
    traceback.print_exc()

INFO:dark_vessel_processor:Parsed manifest: 6 measurement files, 12 calibration files
INFO:dark_vessel_processor:Parsed manifest: 6 measurement files, 12 calibration files


Generated files after updated processing:

slcfile:
  DB_OPENSAR_VD_2_SLC_VH.tiff (262606 bytes)
  DB_OPENSAR_VD_2_SLC_VV.tiff (262606 bytes)

grdfile:

xmlfile:

Testing scene tiling method...


INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC VV tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VV.tiff
INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC VV tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VV.tiff
INFO:dark_vessel_processor:Saved SLC VH tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VH.tiff
INFO:dark_vessel_processor:Saved SLC VV tile: /Data_large/marine/PythonProjects/SAR/sarpyx/data/xview3/output/slcfile/DB_OPENSAR_VD_1_SLC_VV.tiff
INFO:dark_vessel_processor:No vessel data provided for tile 1, skipping metadata generation
INFO:dark_vessel_processor:Gener